## Modify vpa_coco dataset to fit COCO_API

The annotated vpa dataset is formatted on a per-image basis while COCO API requires instance-level annotation (each object in every image is stored as a data entry). Thus we need to reformat vpa_coco to train DETR properly.

In [22]:
%matplotlib inline
from pycocotools.coco import COCO
import pycocotools.mask as maskUtils
import numpy as np
import json

ModuleNotFoundError: No module named 'pycocotools'

In [3]:
import json

In [7]:
dataDir='../../vpa_coco/annotations'
annFile='{}/val2017.json'.format(dataDir)
new_name = '{}/val2017_new.json'.format(dataDir)
annFile = json.load(open('val2017.json'))

In [15]:
st = annFile['stats']
print('stats keys: ', st.keys(), '\n')
pr = st['present_attr']
attr_id = 1
categories = []
attr_to_id = {}
for attr in pr:
    attr_to_id[attr] = attr_id
    categories.append({'id': attr_id, 'name': attr})
    attr_id += 1
print(categories)
print(attr_to_id)
annotations = annFile['annotations']
k = list(annotations.keys())
ann = annotations[k[0]]
print(ann.keys(), '\n')

a = ann['attributes'][0]
print(a.keys(), '\n')
print(a['attr_id'], '\n')
print(a['instance_id'], '\n')

stats keys:  dict_keys(['attr_id_to_n_inst', 'present_attr', 'attr_id_to_n_img', 'n_attr', 'n_images']) 

[{'id': 1, 'name': 'a105_face_all'}, {'id': 2, 'name': 'a106_address_current_all'}, {'id': 3, 'name': 'a107_address_home_all'}, {'id': 4, 'name': 'a108_license_plate_all'}, {'id': 5, 'name': 'a109_person_body'}, {'id': 6, 'name': 'a110_nudity_all'}, {'id': 7, 'name': 'a111_name_all'}, {'id': 8, 'name': 'a18_ethnic_clothing'}, {'id': 9, 'name': 'a24_birth_date'}, {'id': 10, 'name': 'a26_handwriting'}, {'id': 11, 'name': 'a29_ausweis'}, {'id': 12, 'name': 'a30_credit_card'}, {'id': 13, 'name': 'a31_passport'}, {'id': 14, 'name': 'a32_drivers_license'}, {'id': 15, 'name': 'a33_student_id'}, {'id': 16, 'name': 'a35_mail'}, {'id': 17, 'name': 'a37_receipt'}, {'id': 18, 'name': 'a38_ticket'}, {'id': 19, 'name': 'a39_disability_physical'}, {'id': 20, 'name': 'a43_medicine'}, {'id': 21, 'name': 'a49_phone'}, {'id': 22, 'name': 'a70_education_history'}, {'id': 23, 'name': 'a73_landmark'}, {

In [17]:
def save_new(annFile, new_name):
    ca = annFile['created_at']
    st = annFile['stats']
    annotations = annFile['annotations']

    id_count = 0
    new_annotations = []
    images = []
    for k, v in annotations.items():
        img_id = k[5:]
        if v['image_path'][-4] == '.':
            img_path = k + v['image_path'][-4:]
#             print(img_path)
        else:
            print('error')
            break
#         input('enter')
        img = {'id': int(img_id),
               'file_name': img_path,
               'height': v['image_height'], 
               'width': v['image_width']}
        images.append(img)
        
        for inst in v['attributes']:
            inst_anno = {'segmentation': inst['polygons'],
                         'area': inst['area'],
                         'iscrowd': inst['iscrowd'],
                         'image_id': int(img_id),
                         'bbox': inst['bbox'],
                         'category_id': attr_to_id[inst['attr_id']],
                         'id': id_count}
            id_count += 1
            new_annotations.append(inst_anno)
    print(len(new_annotations))

    new_annFile = {'images': images, 'categories':categories, 'annotations':new_annotations}
    with open(new_name, "w") as f:
        json.dump(new_annFile, f)

In [7]:
for name in ['train', 'val', 'test']:
    dataDir='../../vpa_coco/annotations'
    annFile='{}/{}2017.json'.format(dataDir, name)
    new_name = '{}/{}2017_new.json'.format(dataDir, name)
    annFile = json.load(open(annFile))
    save_new(annFile, new_name)

21489
9710
17647


### Load our modified vpa_coco to check

In [21]:
dataDir='../../vpa_coco/annotations'
annFile='{}/train2017_new.json'.format(dataDir)
# annFile = json.load(open(annFile))
# images = annFile['images']
# img = images[1]
# print('images keys: ', img.keys(), '\n')
# print(img)
coco = COCO(annFile)

NameError: name 'COCO' is not defined

In [70]:
k = list(sorted(coco.imgs.keys()))
print(len(k))
img_id = k[0]
path = coco.loadImgs(img_id)[0]['file_name']
print(path)

3873
2017_10001018.jpg


### COCO annotation format

In [2]:
coco_annFile='../../annotations/instances_train2017.json'
coco_annFile = json.load(open(coco_annFile))
print(type(coco_annFile))
print(coco_annFile.keys())

<class 'dict'>
dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])


### Annotations/Category/Images keys

In [3]:
annotations = coco_annFile['annotations']
cate = coco_annFile['categories']
images = coco_annFile['images']

cate = cate[1]
print('category keys: ', cate.keys(), '\n')

ann = annotations[1]
print('annotations keys: ', ann.keys())
print(ann['image_id'])
print(ann['id'], '\n')


img = images[1]
print('images keys: ', img.keys(), '\n')
print(img)

category keys:  dict_keys(['supercategory', 'id', 'name']) 

annotations keys:  dict_keys(['segmentation', 'area', 'iscrowd', 'image_id', 'bbox', 'category_id', 'id'])
200365
509 

images keys:  dict_keys(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id']) 

{'license': 4, 'file_name': '000000522418.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000522418.jpg', 'height': 480, 'width': 640, 'date_captured': '2013-11-14 11:38:44', 'flickr_url': 'http://farm1.staticflickr.com/1/127244861_ab0c0381e7_z.jpg', 'id': 522418}


In [49]:
annFile='../../annotations/instances_train2017.json'
coco = COCO(annFile)

loading annotations into memory...
Done (t=16.82s)
creating index...
index created!


In [56]:
k = list(sorted(coco.imgs.keys()))
img_id = k[10000]
path = coco.loadImgs(img_id)[0]['file_name']
print(path)

000000048795.jpg
